In [1]:
from csv import DictReader

In [2]:
locationFiles = []

with open("formatted_data/locations_final.csv", "r", encoding="utf-8") as locationsCsvFile:
    reader = DictReader(locationsCsvFile)
    for row in reader:
        locationFiles.append(row["ref"] +".csv")

In [3]:
monthCounts = []
monthCountsLookup = {}

for locationFile in locationFiles:
    print(locationFile)
    with open(f"formatted_data/{locationFile}", "r", encoding="utf-8") as locationCsvFile:
        reader = DictReader(locationCsvFile)
        for row in reader:
            year = row["yyyy"]
            month = row["mm"]
            monthId = f"{year}-{month}"
            if monthId in monthCountsLookup:
                monthCounts[monthCountsLookup[monthId]]["COUNT"] += 1
                monthCounts[monthCountsLookup[monthId]]["STATIONS"].append(locationFile.split("data.")[0])
            else:
                monthCountsLookup[monthId] = len(monthCounts)
                monthCounts.append({
                    "YEAR": int(year),
                    "MONTH": int(month),
                    "DISPLAY": monthId,
                    "COUNT": 1,
                    "STATIONS": [locationFile.split("data.")[0]]
                })

monthCounts.sort(key=lambda x: (x["YEAR"], x["MONTH"]))

aberporthdata.csv
armaghdata.csv
ballypatrickdata.csv
bradforddata.csv
braemardata.csv
cambornedata.csv
cambridgedata.csv
cardiffdata.csv
chivenordata.csv
cwmystwythdata.csv
dunstaffnagedata.csv
durhamdata.csv
eastbournedata.csv
eskdalemuirdata.csv
heathrowdata.csv
hurndata.csv
lerwickdata.csv
leucharsdata.csv
lowestoftdata.csv
manstondata.csv
nairndata.csv
newtonriggdata.csv
oxforddata.csv
paisleydata.csv
ringwaydata.csv
rossonwyedata.csv
shawburydata.csv
sheffielddata.csv
southamptondata.csv
stornowaydata.csv
suttonboningtondata.csv
tireedata.csv
valleydata.csv
waddingtondata.csv
whitbydata.csv
wickairportdata.csv
yeoviltondata.csv


In [5]:
print(monthCounts)

[{'YEAR': 1853, 'MONTH': 1, 'DISPLAY': '1853-1', 'COUNT': 2, 'STATIONS': ['armagh', 'oxford']}, {'YEAR': 1853, 'MONTH': 2, 'DISPLAY': '1853-2', 'COUNT': 2, 'STATIONS': ['armagh', 'oxford']}, {'YEAR': 1853, 'MONTH': 3, 'DISPLAY': '1853-3', 'COUNT': 2, 'STATIONS': ['armagh', 'oxford']}, {'YEAR': 1853, 'MONTH': 4, 'DISPLAY': '1853-4', 'COUNT': 2, 'STATIONS': ['armagh', 'oxford']}, {'YEAR': 1853, 'MONTH': 5, 'DISPLAY': '1853-5', 'COUNT': 2, 'STATIONS': ['armagh', 'oxford']}, {'YEAR': 1853, 'MONTH': 6, 'DISPLAY': '1853-6', 'COUNT': 2, 'STATIONS': ['armagh', 'oxford']}, {'YEAR': 1853, 'MONTH': 7, 'DISPLAY': '1853-7', 'COUNT': 2, 'STATIONS': ['armagh', 'oxford']}, {'YEAR': 1853, 'MONTH': 8, 'DISPLAY': '1853-8', 'COUNT': 2, 'STATIONS': ['armagh', 'oxford']}, {'YEAR': 1853, 'MONTH': 9, 'DISPLAY': '1853-9', 'COUNT': 2, 'STATIONS': ['armagh', 'oxford']}, {'YEAR': 1853, 'MONTH': 10, 'DISPLAY': '1853-10', 'COUNT': 2, 'STATIONS': ['armagh', 'oxford']}, {'YEAR': 1853, 'MONTH': 11, 'DISPLAY': '1853-11